## Gap Data, Part 2 

#### Data Documentation:
<br>**Description**: Synthetic dataset from Gap Inc., representing a random sample of individual purchases from Q1 FY2020. <br>Each row is a unique item purchased in an order
<br><br>

| **Feature** | **Description**    | **Sample Value(s)**  |
| ------- | -----------    | ------------- |
| OrderID | Unique identifier per transaction (7-digit) | DRW7C20   |
| CustomerID | Unique identifier per customer (5-digit) | KP441   |
| ProductID  | Unique identifier per item (8-digit) | 13-817-239 |
| StoreID | Unique identifier per store (4-digit) | #4176 |
| OrderType | How purchase was completed  | InStore, HomeDelivery, Online |
| Timestamp | Timestamp of transaction (YYYY-MM-DD) | 2020-01-18 10:13:56	 |
| Brand | Which reporting segment of Gap Inc. bought from | Banana Republic |
| ItemSize | Size of item | XS, S, M, L, X, XL |
| ProductName | Name of item associated with item identifier | Pink Polo by Kanye |
| Collection | Which part of store | Denim Shop |
| Price | Listed price of item | $29.95 |
| ClearanceType | Type of clearance | Retail, Clearance, Final Sale |
| DiscountType | If Gap Card rewards was used | Reward points, Promotion, GapCash, Other |
| StoreName | Store name (i.e. Mall), or facility where online order was shipped from | Barracks Shopping Center |
| Address | Store location | Charlottesville, VA |
| Lat | Store latitude | 38.054034 |
| Lon | Store longitude | -78.499492 |

<br>


One common task that data folks have to do is **merge data from various sources**.

<bR>For example, it'd be a pretty big waste of space on a database to keep listing all the attributes of a store for each item, if we could store them seperately and link them back up when we need to.

<br>This kind of file is sometimes known as a **crosswalk** or **lookup table**. We have individual observations (Gap sales data), and we want to link it up to information about the store.

### Data & Package Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
sales = pd.read_csv('gap_sales.csv', sep='|')
store_lookup = pd.read_csv('gap_stores.csv', sep='|')
locations = pd.read_csv('gap_store_locations.csv', sep='|')

## Store Data

### Fill Missing Values

If you look carefully at the stores, you'll notice that for both the Gap and Banana Republic stores, the StoreName value is empty.

<br>Turns out, there was a mistake made when correcting a typo in the database, and now all stores for 'Williamsburg Premium Outlets' are showing missing. 

1. Fill in the missing value for **store name** (not whole dataframe)
3. Double check that stores #1812 and #4479 no longer have a missing value for either field

In [3]:
store_lookup['StoreName'] = store_lookup.StoreName.fillna('Williamsburg Premium Outlets') #1 
store_lookup[store_lookup.StoreID.isin(['#1812','#4479'])] #2

,Brand,StoreID,StoreName
3,Banana Republic,#4479,Williamsburg Premium Outlets
22,Gap,#1812,Williamsburg Premium Outlets


### Merge to Location Data

1. Take a look at both the store ID lookup, as well as the locations. Mentally note if the sizes are different
2. Consolidate both into one dataframe representing each store (still differentiating by brand)
3. Since one of the keys is now redundant, drop the other column to keep `StoreName`

**Make sure to:** (a) Note which column to merge on, (b) Determine how to do the join logic (i.e. inner? left?)

In [36]:
stores = pd.merge(store_lookup, locations, left_on='StoreName',right_on='Store',how='left',sort=True).drop(columns='Store')
stores.head(3)

,Brand,StoreID,StoreName,Lat,Lon,Address
0,Banana Republic,#6569,Barracks Road Shopping Center,38.054034,-78.499492,"Barracks Road Shopping Center, The Meadows, Ch..."
1,Gap,#9901,Charlottesville Fashion Square,38.077009,-78.474995,"Charlottesville Fashion Square, East Rio Road,..."
2,Banana Republic,#9033,Dulles Town Center,39.027745,-77.419950,"Dulles Town Center, Loudoun County, Virginia, ..."


## Sales Data

Now that we have the store data cleaned up, let's merge it back onto the main dataset.

1. Merge the sales back onto stores, paying attention to the join logic, as well as the better key on which to align the datasets (Hint: its NOT StoreName)
2. Drop the address column, as well as one of the duplicated Brand columns

In [38]:
df = pd.merge(sales, stores.drop(columns=['Address','Brand']), on='StoreID',how='left')
df.sample()

,OrderID,CustomerID,ProductID,StoreID,OrderType,Timestamp,Brand,ItemSize,ProductName,Collection,Price,ClearanceType,StoreName,Lat,Lon
1104,WW1F4ZQ,RO256,64-360-768,#9901,InStore,2020-02-10 19:46:15,Gap,XS,Lightly Used Boxer Briefs,Accessories,9.97,FinalSale,Charlottesville Fashion Square,38.077009,-78.474995
